<h1>Chapter 3 - Looking Inside Transformer LLMs</h1>
<i>An extensive look into the transformer architecture of generative LLMs</i>

This notebook is for Chapter 3 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).


You should use a GPU to run the examples in this notebook. In Google Colab, go to

**Runtime > Change runtime type > Hardware accelerator > GPU > T4 GPU.**

Uncomment and run the following codeblock to install the dependencies.


In [2]:
%%capture
!pip install transformers>=4.41.2 accelerate>=0.31.0

# Loading the LLM

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

# The Inputs and Outputs of a Trained Transformer LLM

Text generation models are autoregressive, meaning they generate one token at a time and feed it back into itself to gnerate the next token. This distinguishes them from text _representation_ models like BERT. The following output stops at 50 tokens because of our `max_new_tokens` setting.

In [4]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)

print(output[0]['generated_text'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in


In addition to this looping two key internal components of the forward pass system are the tokenizer and the language modeling head (LM head). The tokenizer tokenizes the input and passes it into a stack of transformer blocks that process it. From there, the processed data passes into the LM head and receives probability scores for what the next token most likely is.

If the tokenizer has a vocabulary of 50K tokens, then the stack of transformer blocks will have a vector representation associated with each of teh 50K tokens. The LM head spit out "Dear" because it had the highest token probability. The LM head is a simple neural network layer. There are other types of heads: sequence classification heads, and token classification heads.

Print the model to see the nested layers. `lm_head` is the last layer. Notice how Phi3Model has 32,064 tokens each with a 3,072 length embedding vector. After the `embed_dropout` later (skip this for now) is 32 blocs of type `Phi3DecoderLayer` each containing an attention later and a feedforward. The `lm_head` layer receives a 3,072 length embedding vector and outputs a vector or probability scores with length 32,064.


In [5]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

# Choosing a single token from the probability distribution (sampling / decoding)

The method of choosing a single token from this vector is called the _decoding strategy_. One strategy is to select the highest probability score (_greedy decoding_). This is what happens when temperature is set to 0. Another is to sample from the vocabulary using the probabilities as weights.

The next code block gets the model output prior to LM head. `model_output` has shape \[1, 6, 3072\] where 3072 is the embedding vector. `lm_head_output` has shape \[1, 6, 32064\] where 32064 is the model vocabulary size.

In [6]:
prompt = "The capital of France is"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Tokenize the input prompt
input_ids = input_ids.to("cuda")

# Get the output of the model before the lm_head
model_output = model.model(input_ids)

# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])

In [7]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [8]:
lm_head_output.shape

torch.Size([1, 5, 32064])

Index 0 is the batch dimension, -1 is the last token in the sequence. `argmax` gets the top scoring ID.

In [9]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

# Speeding up generation by caching keys and values

Some of the speed from transformer models comes from the initial tokenization step. Each token individually runs through the tansformer blocks, and the tokens can go in parallel. Actually, the model's context length sets a limit on this, but that's still maybe 4K.

More time savings are gained from caching keys and values in the transformer blocks. The difference is illustrated below.

In [10]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

Notice the time difference with and without caching.

In [11]:
%%timeit -n 1
# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=True
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


6.21 s ± 2.22 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit -n 1
# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=False
)

33.9 s ± 913 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
